In [26]:
%%writefile datatest.py
import streamlit as st
import pandas as pd
import plotly.express as px
import  altair as alt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#---------------------------------------------------------------------------------------------------------------------------------------

import mysql.connector as sql

mydb = sql.connect(
  host="localhost",
  user="root",
  password="bawadharani",
  database='phone_pe'
)
print(mydb)
mycursor = mydb.cursor(buffered=True)

# Fetching datas from Mysql:
mycursor.execute("SELECT * FROM Agg_Transaction_Table")
df=mycursor.fetchall()
mycursor.execute("SELECT * FROM Longitude_Latitude_State_Table")
state=mycursor.fetchall()
mycursor.execute("SELECT * FROM Districts_Longitude_Latitude_Table")
districts=mycursor.fetchall()
mycursor.execute("SELECT * FROM district_map_transaction_table")
districts_trans=mycursor.fetchall()
mycursor.execute("SELECT * FROM district_map_registering_table")
app_opening=mycursor.fetchall()
mycursor.execute("SELECT * FROM user_by_device")
user_device=mycursor.fetchall()
#mydb.close()

state = pd.DataFrame(state, columns = ['id','code', 'Latitude', 'Longitude', 'State'])
df = pd.DataFrame(df, columns = ['id', 'State', 'Year', 'Quarter', 'Transaction_type', 'Transaction_count', 'Transaction_amount'])
districts =pd.DataFrame(districts, columns = ['id','State', 'Latitude', 'Longitude', 'District'])
districts_trans = pd.DataFrame(districts_trans, columns = ['id','State', 'Year', 'Quarter', 'District', 'Transaction_count', 'Transaction_amount'])
app_opening = pd.DataFrame(app_opening, columns = ['id' ,'State', 'Year', 'Quarter', 'District', 'Registered_user', 'App_opening'])
user_device = pd.DataFrame(user_device, columns = ['id','State', 'Year', 'Quarter', 'Brand', 'Brand_count', 'Brand_percentage'])

state = state.sort_values(by='State')
state = state.reset_index(drop=True)
df2 = df.groupby(['State']).sum()[['Transaction_count', 'Transaction_amount']]
df2 = df2.reset_index()

choropleth_data = state.copy()

for column in df2.columns:
    choropleth_data[column] = df2[column]
choropleth_data = choropleth_data.drop(labels='State', axis=1)

df.rename(columns={'State': 'state'}, inplace=True)
sta_list = ['andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
            'assam', 'bihar', 'chandigarh', 'chhattisgarh',
            'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
            'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
            'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
            'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
            'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
            'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
            'uttarakhand', 'west-bengal']
state['state'] = pd.Series(data=sta_list)
state_final = pd.merge(df, state, how='outer', on='state')
districts_final = pd.merge(districts_trans, districts,
                           how='outer', on=['State', 'District'])


st.balloons()
with st.container():
    st.title(':violet[PhonePe Pulse Data Visualization]')
    st.write(' ')
    st.subheader(
        ':violet[PhonePe App Users:]')
    st.write(' ')
    scatter_year = st.selectbox('Please select the Year',
                                ('2018', '2019', '2020', '2021', '2022'))
    st.write(' ')
    scatter_state = st.selectbox('Please select State', ('andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
                                                         'assam', 'bihar', 'chandigarh', 'chhattisgarh',
                                                         'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
                                                         'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
                                                         'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
                                                         'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
                                                         'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
                                                         'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
                                                         'uttarakhand', 'west-bengal'), index=10)
    scatter_year = int(scatter_year)
    scatter_reg_df = app_opening[(app_opening['Year'] == scatter_year) & (app_opening['State'] == scatter_state)]
    scatter_register = px.scatter(scatter_reg_df, x="District", y="Registered_user",
                                  hover_name="District", hover_data=['Year', 'Quarter', 'App_opening'], size_max=60)
    st.plotly_chart(scatter_register)
    st.write(' ')

# ------------------------------------- Streamlit Tabs for various analysis -----------------------------------------------------------------
geo_analysis, Device_analysis, payment_analysis, transac_yearwise, overall_analysis = st.tabs(
     ["Map analysis", "Device analysis", "Analyse on payment type", "Transaction analysis","Overall analysis"])
# ------------------------------------------- Geo-analysis ----------------------------------------------------------------------------------
with geo_analysis:
    st.subheader(':violet[Transaction analysis-:]')
    st.write(' ')
    Year = st.selectbox('Please select the Year',('2018', '2019', '2020', '2021', '2022'), key = 'year_selector')
    st.write(' ')
    Quarter = st.selectbox('Please select the Quarter',('1', '2', '3', '4'))
    st.write(' ')
    districts_final['Year'] = pd.to_numeric(districts_final['Year'])
    districts_final['Quarter'] = pd.to_numeric(districts_final['Quarter'])

    Year = int(Year)
    Quarter = int(Quarter)
    plot_district = districts_final[(districts_final['Year'] == Year) & (districts_final['Quarter'] == Quarter)]
    
    state_final['Year'] = pd.to_numeric(state_final['Year'])
    state_final['Quarter'] = pd.to_numeric(state_final['Quarter'])

    plot_state = state_final[(state_final['Year'] == Year) & (state_final['Quarter'] == Quarter)]
    plot_state_total = plot_state.groupby(['state', 'Year', 'Quarter', 'Latitude', 'Longitude']).sum().reset_index()
    state_code = ['AN', 'AD', 'AR', 'AS', 'BR', 'CH', 'CG', 'DNHDD', 'DL', 'GA',
                  'GJ', 'HR', 'HP', 'JK', 'JH', 'KA', 'KL', 'LA', 'LD', 'MP', 'MH',
                  'MN', 'ML', 'MZ', 'NL', 'OD', 'PY', 'PB', 'RJ', 'SK', 'TN', 'TS',
                  'TR', 'UP', 'UK', 'WB']
    plot_state_total['code'] = pd.Series(data=state_code)
# ------------------------------------------- Geo-visualization of transacion data -----------------------------------------------------

import plotly.graph_objects as go
import plotly.express as px

# Create the choropleth map
choropleth_data = state_final.groupby(['State','Longitude','Latitude']).sum()
choropleth_data.reset_index(inplace=True)
fig = px.choropleth(
    choropleth_data,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='State',
    color='Transaction_amount',
    color_continuous_scale='ice',
    hover_data=['Transaction_count', 'Transaction_amount']
)
fig.update_geos(
    visible=True,
    projection=dict(type='mercator'),
    lonaxis=dict(range=[65, 100]),
    lataxis=dict(range=[0,35])
)

# Create the scattergeo plots
fig1 = px.scatter_geo(
    plot_district,
    lon='Longitude',
    lat='Latitude',
    color='Transaction_amount',
    size='Transaction_count',
    hover_name='District',
    hover_data=['State', 'Transaction_amount', 'Transaction_count', 'Year', 'Quarter'],
    title='District',
    size_max=20
)
fig1.update_traces(marker={'color': "#d1cb11", 'line_width': 1})

fig2 = px.scatter_geo(
    plot_state_total,
    lon='Longitude',
    lat='Latitude',
    hover_name='state',
    text='code',
    hover_data=['Transaction_count', 'Transaction_amount', 'Year', 'Quarter'],
    color_discrete_sequence =['#14914d'],
    size='Transaction_count'
)

# Add the scattergeo plots as new traces in the choropleth map
fig.add_trace(fig1.data[0])
fig.add_trace(fig2.data[0])

# Set the layout for the combined map
fig.update_layout(height=600, width=800, title='TRANSACTION DATA STATE & DISTRICT WISE')

# Display the map in the Streamlit app
if st.button('Click here to see map clearly'):
    st.plotly_chart(fig)
    
#--------------------------------------------------User-Device-Analysis-----------------------------------------------------------
with Device_analysis:
    st.subheader('TreeMap view of user device Brands and Usage :')   
    pay_year = 'select * from user_by_device'
    user_device_treemap = pd.read_sql(pay_year, mydb)
    
    import plotly.express as px
    import numpy as np
    user_device_treemap['Brand_percentage'] = user_device_treemap['Brand_percentage'].map(lambda x: f"{x*100:.2f}%")
    
#---------------- TreeMap view of user device --------------  
    fig = px.treemap(user_device, path=[px.Constant("Device analyse"),'State', 'Brand','Year','Quarter','Brand_count'],
                     values='Brand_percentage',
                     color='State', hover_data=[ 'Brand_count','Quarter'],
                     color_continuous_scale='RdBu',
                     color_continuous_midpoint=np.average(user_device['Brand_count'], weights=user_device['Brand_count']))
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    st.plotly_chart(fig)
# ----------------------------------------- Payment type analysis of Transacion data ----------------------------------------------------------
with payment_analysis:
    st.subheader(':violet[Payment type Analysis -> 2018 - 2022:]')
    aggtrans = 'select * from agg_transaction_table'
    payment_mode = pd.read_sql(aggtrans, mydb)
# ----------------Animated line chart analysis of payment mode -------------------
    fig4 = px.scatter(payment_mode, x="Transaction_amount", y="Transaction_type", animation_frame="Year", animation_group="Transaction_amount",
                     size="Transaction_count", color="Transaction_count", hover_name="Transaction_count", facet_col="Transaction_type",
                     log_x=True, size_max=45)
    st.plotly_chart(fig4)
     
# --------------------------------------- Transacion data analysis statewise ------------------------------------------------------------------
with transac_yearwise:
    st.subheader(':violet[statewise Transaction Analysis:]')
    state_name = st.selectbox('Please select State', ('andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
                                                         'assam', 'bihar', 'chandigarh', 'chhattisgarh',
                                                         'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
                                                         'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
                                                         'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
                                                         'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
                                                         'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
                                                         'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
                                                         'uttarakhand', 'west-bengal'), index=10, key='transac')
    quarter = int(st.radio('Please select the Quarter',
                                   ('1', '2', '3', '4'), horizontal=True, key='quarter'))
    Transaction_type = st.selectbox('Please select the Mode',
                                ('Recharge & bill payments', 'Peer-to-peer payments', 'Merchant payments', 'Financial Services', 'Others'), key='transactype')
    Transaction_values = st.selectbox(
        'Please select the values to visualize', ('Transaction_count', 'Transaction_amount'), key='Transaction values')

    querypay_year = 'select * from agg_transaction_table'
    payment_mode_yearwise = pd.read_sql(querypay_year, mydb)

    new_df = payment_mode_yearwise.groupby(
        ['State', 'Year', 'Quarter', 'Transaction_type']).sum()
    new_df = new_df.reset_index()
    chart = new_df[(new_df['State'] == state_name) &
                   (new_df['Transaction_type'] == Transaction_type) & (new_df['Quarter'] == quarter)]
    
# --------- Bar chart scatter analysis of transacion data statewise ---------------------------

    year_fig = px.scatter(chart, x=['Year'], y=Transaction_values, color=Transaction_values,
                 title='Transaction_Analysis')
    st.plotly_chart(year_fig)
    
# -----------------------------------overall india transacion data  -----------------------------------------------------------------

with overall_analysis: 
    st.subheader(':violet[India Transaction Analysis:]')
    all_values = st.selectbox(
        'Please select the values to visualize', ('Transaction_count', 'Transaction_amount'), key='values')
    all_trans = new_df.groupby(['Year']).sum()
    all_trans.reset_index(inplace=True)
    
    all_trans = px.bar(all_trans, x='Year', y=all_values, color=all_values,
                     title='Transacion of all over India', color_continuous_scale='rdbu')
    all_trans.update_layout(height=500, width=800)
    st.plotly_chart(all_trans)

# -------------------------------India user device analysis --------------------------------------------------------------
    st.subheader('Brand Analysis 2018-2022')
    device_type = 'select * from user_by_device'
    user_device_overall = pd.read_sql(device_type, mydb)
    all_brand = user_device_overall.groupby(['Brand', 'Year']).sum()
    all_brand.reset_index(inplace=True)
    
    circle = px.sunburst(all_brand, path=['Year', 'Brand', 'Brand_count'], values='Brand_percentage')
    circle.update_layout(height = 400,width = 400)
    st.plotly_chart(circle)

    # -------------------------- India registered and app opening --------------------------------------------------------
    reg_user = 'select * from district_map_registering_table'
    overall_reg = pd.read_sql(reg_user, mydb)
    overall_reg = overall_reg.groupby(['State', 'Year']).sum()
    overall_reg.reset_index(inplace=True)

    reg = px.bar(overall_reg, x='Year', y=[
        'Registered_user', 'App_opening'], barmode='group', title='Phonepe Users')
    reg.update_layout(height=500, width=800)
    st.plotly_chart(reg)


Overwriting datatest.py
